In [1]:
import numpy as np
import numpy.random as npr
import math
from scipy import stats
import datetime as dt
import random
import matplotlib.pyplot as plt

In [2]:
#taken from Optiver TraderCraft workshop and competition

_norm_cdf = stats.norm(0, 1).cdf
_norm_pdf = stats.norm(0, 1).pdf


def _d1(S, K, T, r, sigma, div = 0):
    return (np.log(S / K) + (r - div + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))


def _d2(S, K, T, r, sigma, div = 0):
    return _d1(S, K, T, r, sigma) - sigma * np.sqrt(T)


def calculate_time_to_date(expiry_datetime: dt.datetime, current_datetime: dt.datetime):
    """
    Calculate the time until expiration of the option. All options expire at 12:00:00 UTC on the date of expiry.
    
    Example: calculate_time_to_date(dt.datetime.strptime('2021-03-01 12:00:00', '%Y-%m-%d %H:%M:%S'), dt.datetime.now())
    """
    return (expiry_datetime - current_datetime) / dt.timedelta(days=1) / 365


def call_value(S, K, T, r, sigma, div=0):
    return S * np.exp(-div * T) *_norm_cdf(_d1(S, K, T, r, sigma)) - K * np.exp(-r * T) * _norm_cdf(
        _d2(S, K, T, r, sigma)
    )


def put_value(S, K, T, r, sigma, div=0):
    return np.exp(-r * T) * K * _norm_cdf(-_d2(S, K, T, r, sigma)) - S * np.exp(-div * T) * _norm_cdf(
        -_d1(S, K, T, r, sigma)
    )

In [3]:
def call_delta(S, K, T, r, sigma, div=0):
    return  np.exp(-div * T) * _norm_cdf(_d1(S, K, T, r, sigma))

def put_delta(S, K, T, r, sigma, div=0):
    return np.exp(-div * T) *_norm_cdf(_d1(S, K, T, r, sigma)) -np.exp(-div * T)

def call_vega(S, K, T, r, sigma, div=0):
    return S * _norm_pdf(_d1(S, K, T, r, sigma)) * np.sqrt(T) * np.exp(-div * T)

def put_vega(S, K, T, r, sigma, div=0):
    return call_vega(S, K, T, r, sigma)

def call_gamma(S, K, T, r, sigma, div=0):
    return _norm_pdf(_d1(S, K, T, r, sigma)) * np.exp(-div * T) / (S * sigma *np.sqrt(T))

def put_gamma(S, K, T, r, sigma, div=0):
    return call_gamma(S, K, T, r, sigma, div=0)


def call_theta(S, K, T, r, sigma, div=0):
    return -(S * _norm_pdf(_d1(S, K, T, r, sigma)) * sigma)/(2*np.sqrt(T))  - r * K * np.exp(-r * T) * _norm_cdf(_d1(S, K, T, r, sigma))

def put_theta(S, K, T, r, sigma, div=0):
    return call_theta(S, K, T, r, sigma, div=0) + r * K * np.exp(-r * T)

In [4]:
def approx(V_0, S_0, epsilon, delta, gamma = 0, h = 0, theta = 0, vega = 0, delta_vol = 0):
    return V_0 + epsilon*delta*S_0 + gamma*0.5*(epsilon*S_0)**2 + h*theta + vega*delta_vol

In [34]:
S = 42627
K_1 = 38000
K_2 = 30
K_3 = 36
T_1 = 2/365
T_2 = 1/2
r = 0
div = 0
sigma=1.14

In [35]:
put_value(S, K_1, T_1, r, sigma, div)

135.36252718025207

In [8]:
call_vega(S, K_2, T_1, r, sigma, div)

3.6505481739031542

In [9]:
call_vega(S, K_3, T_1, r, sigma, div)

0.005569041930214075

In [13]:
put_vega(S, K_2, T2, r, sigma, div)

14.08261307057198

In [29]:
-1*call_gamma(S, K, T, r, sigma/10, div) -1*put_gamma(S, K, T, r, sigma/10, div)

-0.015952705216655275

In [15]:
-1*call_delta(S, K_1, T, r, sigma, div) -1*put_delta(S, K_2, T, r, sigma, div) -1*call_delta(S, K, T, r, sigma, div) -1*put_delta(S, K, T, r, sigma, div)

-0.07681353809731561

In [25]:
call_gamma(S, K_1, T, r, sigma, div) - call_gamma(S, K_2, T, r, sigma, div)

-0.006611586197486618

In [17]:
call_gamma(1350, 1350, 1/52, r, sigma, div)

0.008877825294447169

In [10]:
call_vega(S, K_2, T, r, sigma, div) + put_vega(S, K_1, T, r, sigma, div)

0.13108507887658355

In [116]:
-10*call_gamma(S, K, T, r, sigma, div) -10* put_gamma(S, K, T, r, sigma, div)

-0.29939962723721464

In [117]:
-10*call_theta(S, K, T, r, sigma, div) -10* put_theta(S, K, T, r, sigma, div)

55.1853392923634

In [120]:
-10*call_vega(S, K, T, r, sigma, div) -10* put_vega(S, K, T, r, sigma, div)

-792.1604235873122

In [72]:
V_0 = 189.86
epsilon = -5/100
delta = 3.1
gamma = 32.97
theta = -375.407
h = 1/365
vega = 791.3156
delta_vol = -1.2/100
approx(V_0, S, epsilon, delta, gamma, h, theta, vega, delta_vol)

192.72070047123287

In [58]:
189.86-5/100 * 20 *3.1 + 32.97*0.5*(5/100*20)**2+ (1/365)*(-375.407)

202.21648767123287

In [26]:
189.86-5/100 * 20 *3.1

186.76000000000002

In [20]:
100*call_delta(S, K, T, r, sigma, div) + 100*put_delta(S, K, T, r, sigma, div) 

8.03736701439017

In [181]:
put_delta(S, K, T, r, sigma)

-0.4979118502019869

In [18]:
100*call_gamma(S, K, T, r, sigma) + 100*put_gamma(S, K, T, r, sigma) 

33.07382025010852

In [183]:
call_vega(S, K, T, r, sigma)

41.76261456443165

In [184]:
put_vega(S, K, T, r, sigma)

41.76261456443165

In [244]:
100*call_theta(S, 2100, T, r, sigma) -10*put_theta(S, 1900, T, r, sigma) 

724.0692239028373

In [245]:
-10*call_vega(S, 2100, T, r, sigma, div=0) - 10*put_vega(S, 1900, T, r, sigma, div=0)

-396.7502596727875